In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#!pip install pymorphy2==0.8
#!pip install pymystem3

import pandas as pd
import regex as re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
#from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Считаем обучающую и тестовую выборки
train_data = pd.read_csv('rusentitweet_train.csv')
test_data = pd.read_csv('rusentitweet_test.csv')

determined_labels = ['positive', 'negative']

filtered_train_data = train_data[train_data['label'].isin(determined_labels)]
print(filtered_train_data.head())

filtered_test_data = test_data[test_data['label'].isin(determined_labels)]
print(filtered_test_data.head())

                                                 text     label  \
0   Помойму я вкрашилась в Чимина🤧 https://t.co/t2...  positive   
5                       @buybread_ я не с порядке!!!!  negative   
10  @ange1flyhigh В следующий раз буду до победног...  positive   
15  @LimitaVIP Удивительный гiмн...\r\nУдивительно...  negative   
17                            я срала на эту биологию  negative   

                     id  
0   1282311169534038016  
5   1335130757044563971  
10  1215370396465291267  
15  1253799540848762887  
17  1339418979887173632  
                                                text     label  \
1      я считаю это мем года https://t.co/xoVKj5y8Mj  positive   
2  ян русский на сотку все запятые где надо🤙🏻👍🏻👍🏻...  positive   
5  (пушка на Караульной горе больше не стреляет Б...  negative   
6      @Iori_loves_U Как мило /смутилась/ спасибо 🥰🌸  positive   
9        мне больно дышать..я как бабка 80 лет ужас🚬  negative   

                    id  
1  1218052288964632

In [ ]:
# Предобработка
def preprocess(text):
    # Удаление ссылок на аккаунты
    text = re.sub(r'^(@\w+\s)+', '', text)

    # Удаление ссылок на твиты
    text = re.sub(r'https://t.co/\w+', '', text)

    # Приведение к нижнему регистру
    text = text.lower()

    # Замена всех "ё" на "е"
    text = text.replace('ё', 'е')

    # Удаление спецсимовлов, кроме !,? и эмодзи
    text = re.sub(r'[^\w\s\p{So}!?]', ' ', text)

    return text

preprocessed_train_data = filtered_train_data.copy()
preprocessed_train_data['text'] = preprocessed_train_data['text'].apply(preprocess)
print(preprocessed_train_data.head())

preprocessed_test_data = filtered_test_data.copy()
preprocessed_test_data['text'] = preprocessed_test_data['text'].apply(preprocess)
print(preprocessed_test_data.head())

                                                 text     label  \
0                     помойму я вкрашилась в чимина🤧   positive   
5                                  я не с порядке!!!!  negative   
10  в следующий раз буду до победного ее закрывать...  positive   
15  удивительный гiмн   \r\nудивительно  что пока ...  negative   
17                            я срала на эту биологию  negative   

                     id  
0   1282311169534038016  
5   1335130757044563971  
10  1215370396465291267  
15  1253799540848762887  
17  1339418979887173632  
                                                text     label  \
1                             я считаю это мем года   positive   
2  ян русский на сотку все запятые где надо🤙 👍 👍 ...  positive   
5   пушка на караульной горе больше не стреляет б...  negative   
6                    как мило  смутилась  спасибо 🥰🌸  positive   
9        мне больно дышать  я как бабка 80 лет ужас🚬  negative   

                    id  
1  1218052288964632

In [ ]:
# Стемминг
def stemming(text):
    stemmer = SnowballStemmer("russian")

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('russian'))
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words] # +удаление стоп-слов
    stemmed_text = ' '.join(stemmed_tokens)

    return stemmed_text

# Лемматизация
def lemmatization(text):
    mystem = Mystem()
    lemmas = mystem.lemmatize(text)

    lemmas = [lemma.strip() for lemma in lemmas if lemma.isalnum()]
    lemmatized_text = ' '.join(lemmas)

    return lemmatized_text

transformed_train_data = preprocessed_train_data.copy()
#transformed_train_data['text'] = transformed_test_data['text'].apply(stemming)
transformed_train_data['text'] = transformed_train_data['text'].apply(lemmatization)
print(transformed_train_data.head())


transformed_test_data = preprocessed_test_data.copy()
#transformed_test_data['text'] = transformed_test_data['text'].apply(stemming)
transformed_test_data['text'] = transformed_test_data['text'].apply(lemmatization)
print(transformed_test_data.head())

In [ ]:
# Преобразование в мешок слов
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(transformed_train_data['text'])
X_test_counts = vectorizer.transform(transformed_test_data['text'])
print(X_train_counts, '\n\n')
print(X_test_counts)

  (0, 4915)	1
  (0, 1005)	1
  (0, 7348)	1
  (1, 4989)	1
  (2, 6028)	1
  (2, 799)	1
  (2, 4649)	1
  (2, 2129)	1
  (2, 4797)	1
  (2, 6812)	1
  (3, 6817)	2
  (3, 1365)	1
  (3, 4832)	1
  (3, 5878)	1
  (3, 6847)	1
  (4, 6281)	1
  (4, 652)	1
  (5, 4897)	1
  (5, 404)	1
  (5, 5520)	1
  (5, 1350)	1
  (5, 4673)	1
  (6, 4544)	1
  (6, 7303)	1
  (6, 1086)	1
  :	:
  (4563, 7044)	1
  (4564, 5971)	1
  (4564, 6528)	1
  (4564, 3370)	1
  (4564, 6807)	1
  (4564, 2967)	1
  (4564, 7219)	1
  (4564, 4518)	1
  (4564, 4267)	1
  (4564, 1599)	1
  (4565, 3115)	1
  (4565, 1604)	1
  (4566, 6967)	1
  (4566, 7181)	1
  (4566, 1729)	1
  (4567, 2091)	1
  (4567, 3849)	1
  (4567, 6946)	1
  (4567, 1171)	1
  (4568, 492)	1
  (4568, 6532)	1
  (4568, 3653)	1
  (4568, 5038)	1
  (4568, 2348)	1
  (4568, 6921)	1 


  (0, 1404)	1
  (0, 3263)	1
  (0, 7561)	1
  (1, 5765)	1
  (1, 5938)	1
  (1, 6195)	1
  (1, 6528)	1
  (1, 7365)	1
  (1, 7561)	1
  (1, 7613)	1
  (2, 519)	1
  (2, 1430)	1
  (3, 3307)	1
  (3, 6089)	1
  (3, 6223)	1
  (4, 108)	

In [ ]:
# Создание экземпляра TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Преобразование текста в TF-IDF матрицу для обучающих данных
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Преобразование текста в TF-IDF матрицу для тестовых данных
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

print(X_train_tfidf, '\n\n')
print(X_test_tfidf)

  (0, 7348)	0.5862800063736923
  (0, 4915)	0.5862800063736923
  (0, 1005)	0.5590630628586786
  (1, 4989)	1.0
  (2, 6812)	0.39960587207917836
  (2, 6028)	0.3874703830260081
  (2, 4797)	0.43730214415327695
  (2, 4649)	0.4749984162273756
  (2, 2129)	0.43730214415327695
  (2, 799)	0.2865170558568824
  (3, 6847)	0.3339688054438098
  (3, 6817)	0.7197954718835435
  (3, 5878)	0.36731329848813576
  (3, 4832)	0.2737034718166318
  (3, 1365)	0.4006577915324617
  (4, 6281)	0.7291126923545203
  (4, 652)	0.6843936599995228
  (5, 5520)	0.38995284350726117
  (5, 4897)	0.455849472563417
  (5, 4673)	0.47804165475242977
  (5, 1350)	0.47804165475242977
  (5, 404)	0.42789062953910717
  (6, 7383)	0.2849251350148732
  (6, 7303)	0.2912726381861287
  (6, 6709)	0.41708449874405545
  :	:
  (4563, 1897)	0.27216802024634523
  (4564, 7219)	0.34707989812458057
  (4564, 6807)	0.3542313537634815
  (4564, 6528)	0.27241383860907425
  (4564, 5971)	0.25680400622549443
  (4564, 4518)	0.373041946989584
  (4564, 4267)	0.38638

In [ ]:
label_mapping = {"positive": 1, "negative": 0}

# Преобразование меток в численный формат
y_train_encoded = transformed_train_data['label'].map(label_mapping)
y_test_encoded = transformed_test_data['label'].map(label_mapping)

# Обучение логистической регрессии
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train_tfidf, y_train_encoded)

# Обучение случайного леса
random_forest = RandomForestClassifier()
random_forest.fit(X_train_tfidf, y_train_encoded)

y_train_pred_lr = logistic_regression.predict(X_train_tfidf)
y_train_pred_rf = random_forest.predict(X_train_tfidf)

y_test_pred_lr = logistic_regression.predict(X_test_tfidf)
y_test_pred_rf = random_forest.predict(X_test_tfidf)

RandomForestClassifier()

In [ ]:
# Обучающая выборка
print('Обучающая выборка\n')

# Лог. регрессия
print('Лог. регрессия\n')
print('Accuracy = ', accuracy_score(y_train_encoded, y_train_pred_lr))
print('ROC-AUC = ', roc_auc_score(y_train_encoded, y_train_pred_lr))
print('Precision = ', precision_score(y_train_encoded, y_train_pred_lr))
print('Recall = ', recall_score(y_train_encoded, y_train_pred_lr))
print('F1-мера = ', f1_score(y_train_encoded, y_train_pred_lr))

# Случайный лес
print('\nСлучайный лес\n')
print('Accuracy = ', accuracy_score(y_train_encoded, y_train_pred_rf))
print('ROC-AUC = ', roc_auc_score(y_train_encoded, y_train_pred_rf))
print('Precision = ', precision_score(y_train_encoded, y_train_pred_rf))
print('Recall = ', recall_score(y_train_encoded, y_train_pred_rf))
print('F1-мера = ', f1_score(y_train_encoded, y_train_pred_rf))


# Тестовая выборка
print('\n\nТестовая выборка\n')

# Лог. регрессия
print('Лог. регрессия\n')
print('Accuracy = ', accuracy_score(y_test_encoded, y_test_pred_lr))
print('ROC-AUC = ', roc_auc_score(y_test_encoded, y_test_pred_lr))
print('Precision = ', precision_score(y_test_encoded, y_test_pred_lr))
print('Recall = ', recall_score(y_test_encoded, y_test_pred_lr))
print('F1-мера = ', f1_score(y_test_encoded, y_test_pred_lr))

# Случайный лес
print('\nСлучайный лес\n')
print('Accuracy = ', accuracy_score(y_test_encoded, y_test_pred_rf))
print('ROC-AUC = ', roc_auc_score(y_test_encoded, y_test_pred_rf))
print('Precision = ', precision_score(y_test_encoded, y_test_pred_rf))
print('Recall = ', recall_score(y_test_encoded, y_test_pred_rf))
print('F1-мера = ', f1_score(y_test_encoded, y_test_pred_rf))

coefficients = logistic_regression.coef_[0]
indices_lr = abs(coefficients).argsort()[::-1]
feature_names = vectorizer.get_feature_names_out()

top_n = 20
top_features_lr = [feature_names[index] for index in indices_lr[:top_n]]
print(f"\nTop {top_n} Important Features (Words) for Logistic Regression:")
print(top_features_lr)

feature_importances = random_forest.feature_importances_
indices_rf = feature_importances.argsort()[::-1]
top_features_rf = [feature_names[index] for index in indices_rf[:top_n]]
print(f"\nTop {top_n} Important Features (Words) for Random Forest:")
print(top_features_rf)

Обучающая выборка

Лог. регрессия

Accuracy =  0.9150798861895382
ROC-AUC =  0.9031424949224359
Precision =  0.9684274438372799
Recall =  0.8259968928016572
F1-мера =  0.8915595304639463

Случайный лес

Accuracy =  0.9949660757277303
ROC-AUC =  0.9955712411792905
Precision =  0.9887295081967213
Recall =  0.9994821336095288
F1-мера =  0.9940767447849601


Тестовая выборка

Лог. регрессия

Accuracy =  0.7637795275590551
ROC-AUC =  0.7332674571805007
Precision =  0.8491803278688524
Recall =  0.5362318840579711
F1-мера =  0.6573604060913706

Случайный лес

Accuracy =  0.7165354330708661
ROC-AUC =  0.7167890080933559
Precision =  0.6485981308411215
Recall =  0.7184265010351967
F1-мера =  0.6817288801571709

Top 20 Important Features (Words) for Logistic Regression:
['любл', 'блят', 'красив', 'пиздец', 'лучш', 'крут', 'классн', 'прекрасн', 'мил', 'нах', 'сук', 'хорош', 'рад', 'ва', 'нрав', 'любим', 'хуйн', 'обожа', 'вообщ', 'любов']

Top 20 Important Features (Words) for Random Forest:
['люб